In [2]:
import pandas as pd
import csv,gc,copy
import numpy as np

In [168]:
%%HTML
<button onclick="$('.input, .prompt, .output_stderr, .output_error, .output_result').toggle();">Toggle Code</button>

In [3]:
def DetecteAbnormalValue(dfseries,nagetive='fillzero',unwantedstr=0,dtype=False,fillna="dontfill"):
    
    #unwanted str
    if unwantedstr !=False:
    
        dfseries=pd.to_numeric(dfseries,errors='coerce')
        
    else:
        pass
    #negative value
    if nagetive =='fillzero':
        abnormal_nagtive=[i for i in set(dfseries) if i <0]
        if abnormal_nagtive != []:
            dfseries.replace(abnormal_nagtive,0)
        else:
            pass
        
    elif nagetive == 'abs':
        dfseries = abs(dfseries)
        
    else:
        pass
    
    if fillna == "dontfill":
        pass
        
    elif fillna=='mean':
        dfseries=dfseries.fillna(dfseries.mean())
    elif fillna=='max':
        dfseries=dfseries.fillna(dfseries.max())
    else:
        dfseries=dfseries.fillna(fillna)
    
    
    
    if dtype != False:
        dfseries=pd.to_numeric(dfseries,errors='coerce')
        dfseries=dfseries.astype(dtype)
    else:
        pass
    return dfseries


def GroupDatebyStates(AA):
    
    
    list_state={}
    group_state={}

    
    for state in states:
        locals()['A%s'%state]=AA[AA['Current State/Province of Incorporation Code']==state]
        temp =AA[AA['Current State/Province of Incorporation Code']==0]
        locals()['temp%s'%state] =temp[temp['State/Province']==state]
        locals()['%s'%state] =pd.concat([locals()['A%s'%state],locals()['temp%s'%state]],ignore_index=True)
        years =list(set(locals()['%s'%state]['Data Year - Fiscal']))
        if 0.0 in years:
            years =years[1:]
        else:
            pass
        GroupUpVariableYearList =GroupUpVariableByYear(locals()['%s'%state],years)
        group_state.update({state:GroupUpVariableYearList})
        
        list_state.update({state: {"DateFrame":locals()['%s'%state],"Ticker in This State":set(locals()['%s'%state]['Ticker Symbol'])}})
        
    return list_state,group_state
def CalcualteRDM(AA):
    if 'Price Close - Annual - Calendar' in AA.columns:
        AA['MRKTVALUE']=AA['Price Close - Annual - Calendar']*AA['Common Shares Outstanding']
        AA['RD/MarketValue'] = AA['Research and Development Expense']/AA['MRKTVALUE']
        AA['RD/MarketValue']=AA['RD/MarketValue'].replace([np.inf, -np.inf], np.nan)
    else:
        AA['RD/MarketValue']='ERROR'
        
    return AA
def GroupUpVariableByYear(stateDataframe,years):
    GroupUpVariableYearList={}
    total=set([])

    for year in years:
        target_df=stateDataframe[stateDataframe['Data Year - Fiscal']==year]
        target_df =target_df.sort(["RD/MarketValue"],ascending=True)
        if target_df.shape[0]<=5:
            top = set(target_df.head(1)['Ticker Symbol'])
            bottom = set(target_df.tail(1)['Ticker Symbol'])
            #choose top one and bottom one
        elif target_df.shape[0]>5:
            top = set(target_df.head(target_df.shape[0]/5)['Ticker Symbol'])
            bottom = set(target_df.tail(target_df.shape[0]/5)['Ticker Symbol'])
            
        else:
            top =set([])
            bottom =set([])
            pass
        total =total|top|bottom
        GroupUpVariableYearList.update({year:{"top":top,"bottom":bottom}})
    GroupUpVariableYearList.update({"total":total})
    return GroupUpVariableYearList
def CalculateReturnAll(return_file):
    tickers =list(set(return_file.TICKER))[1:]
    df_empty = pd.DataFrame(columns=['date','Return','PRC','SHROUT','TICKER']) 
    for ticker in tickers:
        temp =return_file[return_file['TICKER']==ticker]
        temp['Return']=temp['PRC'].shift(1)-temp['PRC']/temp['PRC']
        df_empty = pd.concat(df_empty,temp[['date','Return','PRC','SHROUT','TICKER']],ignore_index=True)
    return df_empty
def CalculateReturn(temp):
    temp['Return']=temp['PRC'].shift(1)-temp['PRC']/temp['PRC']
    return temp
def TrandsformYMD(ss):
    [month,day,year]=ss.strip().split("/")
    if int(month)<10:
        month =str(0)+str(int(month))
    else:
        pass
    if int(day)<10:
        day =str(0)+str(int(day))
    else:
        pass
    return year+month+day 
def TransformDataFrame(w):
    collected=[]
    for information in list(w.T[list(w.T.columns)[0]]):
        #print information
        
        if  information==0:
            collected.append((0,0))
        elif len(information)==2:
            collected.append((information,"State"))
        elif isinstance(information,str):
            splitout =[x for x in information.split(' ') if x !='']
            #print splitout
            date =TrandsformYMD(splitout[0])
            if len(splitout)==1:
                collected.append((int(date),"NOW"))
            elif len(splitout)==2:
                
                try:
                    tttemp =int(splitout[1])
                
                    collected.append((int(date),tttemp))
                
                except Exception as e:
                    #print splitout[1]
                    collected.append((int(date),splitout[1]))
            
                    
            elif len(splitout)>=3 and splitout[1] =="(Y)" and len(splitout[2:])==1:
                #print int(TrandsformYMD(splitout[2].split("(")[0]))

                collected.append((int(date),int(TrandsformYMD(splitout[2].split("(")[0]))))
            elif len(splitout)>=3  and len(splitout[2:])>1:
                t=[(int(date),"Y")]

                for date2_str in splitout[2:]:


                    if isinstance(date2_str,str):

                        if "(" in date2_str:
                            date2 = TrandsformYMD(date2_str.split("(")[0])


                            date2_status=date2_str.split("(")[1].strip(")")

                            te =(int(date2),date2_status)

                            t.append(te)
                    else:
                        pass

                collected.append(t)


    return collected 
def findloaction(testdata,date):
    temP_date2 =copy.deepcopy(date)
    temP_date2.insert(0,testdata)
    countstart = sorted(temP_date2).index(testdata)
    return countstart

def returendict(a):
    a_dict={}
    for aa in a:
        a_dict.update({aa[0]:aa})
    return a_dict
    

def returnvalue(target,a_dict):
    test = a_dict.keys()
    
    test.insert(0,target)
    test.sort()

    return a_dict[sorted(a_dict.keys())[test.index(target)-1]][2]


def periodforlaw(status_dict,date):
        wlist =sorted((status_dict.keys()))
        wlist.extend([sorted(date)[0],sorted(date)[-1]])
        wlist.sort()
        stest =[]
        for i in xrange(len(wlist)):
            if i+1<len(wlist):
                if wlist[i] in sorted((status_dict.keys())):
                    
                    if status_dict[wlist[i]] == "Y":
                        status=1
                    elif status_dict[wlist[i]] == "N":
                        status=0
                    else:
                        status=0
                        
                else:
                    status =0
                    
                stest.append((wlist[i],wlist[i+1],status))
            else:
                pass
        return stest
def RemoveDuplicateValue(l1):
    l2=[]
    for i in l1:
        if not i in l2:
            l2.append(i)
    return l2
def CombineShortValue(r,date):
    dict_r={}
    short_r=RemoveDuplicateValue(r)
    #print short_r
    #print short_r
    for st in short_r:
        
        date_temp=copy.deepcopy(date)
        if isinstance(st,tuple):
            if st[1]=='State':
                temp_return =[st[0]]*datelen
            elif st[1] == 0:
                temp_return=[st[0]]*datelen
            
                
            elif st[1]  == 'NOW' or st[1] == "(Combo)" or st[1] == "(S)" or st[1] == "(Y)":
                testfordate =int(st[0])

                date_temp.insert(0,testfordate)

                count = sorted(date_temp).index(testfordate)
                temp_return =[0]*count+[1]*(datelen-count)
            elif isinstance(st[1],str):
                try:
                
                    gwuhu = int(st[1])
                    if isinstance(gwuhu,int):
                        countstart=findloaction(int(st[0]),date)
                        countend=findloaction(int(st[1]),date)
                        temp_return =[0]*countstart+[1]*(datelen-countstart-countstart)+[0]*countend
                except Exceptation as e:
                    raise 'error:%s'%sr[1] 

                    
                    
                    

        elif isinstance(st,list):
            status_dict={}

            temp_return=[]
            for s in st:
                status_dict.update({s[0]:s[1]})
            period =periodforlaw(status_dict,date)
            a_dict=returendict(period)
            for dates in sorted(date):
                temp_return.append(returnvalue(dates,a_dict))
        else:
            pass
        dict_r.update({st[0]:temp_return})
    return dict_r

In [4]:
data_AA_csv  = 'AA.csv'
data_AB_csv  = 'AB.csv'
data_allacc = 'allacc.csv'
state_csv = 'law_date.csv'

return_csv = 'return.csv'
return2_csv='return2.csv'
return2_file= pd.read_csv(return2_csv, low_memory=False)
return2_file.PRC =abs(return2_file.PRC)
return2_file['TICKER']=return2_file['TICKER'].fillna(0)
return2_file.RET=return2_file.RET.replace("C",np.nan)

In [4]:
AA =pd.read_csv(data_allacc)

states =list(pd.read_csv(state_csv)[u'State'])
AA=AA.fillna(0)
years =list(set(AA['Data Year - Fiscal']))[1:]
return_file= pd.read_csv(return_csv, low_memory=False)
return_file.PRC =abs(return_file.PRC)
return_file['TICKER']=return_file['TICKER'].fillna(0)


return2_file= pd.read_csv(return2_csv, low_memory=False)
return2_file.PRC =abs(return2_file.PRC)
return2_file['TICKER']=return2_file['TICKER'].fillna(0)
#return_file[return_file['TICKER'] !=0]
states
unwnated_state_list=list(set(AA['State/Province'])- set(states))[1:]
#
#calculate RD/Market Value
#RD_date =AA[['Data Date','Data Year - Fiscal','Ticker Symbol','Common Shares Outstanding','Research and Development Expense','Price Close - Annual - Calendar']]

    
    
AA=CalcualteRDM(AA)

#group data by states
list_state,group_state =GroupDatebyStates(AA)


##Calcualate return


/Users/sn0wfree/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:82: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [5]:
list_state["CA"].keys()
group_state["CA"]['total']

{'1910B',
 '2547B',
 '2870B',
 '2912B',
 '2921B',
 '2975B',
 '3044B',
 '3078B',
 '3079B',
 '3095B',
 '3ADRE.1',
 '3ARBC.',
 '3BION.',
 '3BKLO',
 '3BMTRE',
 '3BSURE',
 '3CPTH',
 '3CRTS',
 '3CSVO',
 '3CWICE',
 '3DSCY',
 '3DVGL',
 '3EDWFE',
 '3EGCPQ',
 '3FNNR',
 '3FPMI.',
 '3FSNR.',
 '3GPSCE',
 '3GRUNE',
 '3ICTI',
 '3JNZZ',
 '3KIGAE',
 '3KNICQ',
 '3MCIFE',
 '3MIGA.',
 '3MINSF',
 '3MLNEE',
 '3MSVCE',
 '3NTCRA',
 '3NWKPE',
 '3OISI',
 '3OMCA',
 '3PCAK',
 '3PDSHQ',
 '3PISCE',
 '3PLAZ',
 '3PLHCQ',
 '3PTRH',
 '3PWSPE',
 '3REMC',
 '3RSDL',
 '3SAZZ',
 '3SCMUE',
 '3SEGA',
 '3SGIC',
 '3SKYSE',
 '3SOSDE',
 '3SPOP',
 '3STCHZ',
 '3SUMH',
 '3SWFGE',
 '3SWST',
 '3THCO',
 '3TMKTE',
 '3TONEE',
 '3TTLA',
 '3UNVI',
 '3VIAD',
 '3WDWT',
 '3WNTX',
 '3WWTK',
 '3ZBSTQ',
 '4063B',
 '4078A',
 '4141B',
 '4149B',
 '4190B',
 '4201B',
 '4275B',
 '4321B',
 '4334B',
 '4350B',
 '4431B',
 '4490B',
 '4491B',
 '4492B',
 '4541B',
 '4542B',
 '4620B',
 '5709B',
 '5723B',
 '5856C',
 '6048B',
 '6067B',
 '6182B',
 '6250B',
 '6411

In [7]:
#return
return_short = return2_file[['date','PRC','SHROUT','TICKER','RET']]
#return_dict=CalculateReturn(return_short)
return_short['year']=[int(str(date)[0:4]) for date in list(return_short['date'])]
return_short['MKT']=return_short['SHROUT']* return_short['PRC']   
return_short=return_short.fillna(0)
return_short['RET']=return_short['RET'].replace("B",0)   
return_short['RET']=return_short['RET'].replace("C",0) 
return_short['RET']=return_short['RET'].astype(float)
return_short['MKT']=return_short['MKT'].astype(float)

In [8]:
#for state in group_state.keys():
gc.enable()

temp_dict_forstate=[]
for state in group_state.keys():
#ticker_list=group_state[state]
    for year in group_state[state].keys():
        if year !='total':
            top_list=group_state[state][year]["top"]
            bottom_list=group_state[state][year]["bottom"]
            #print return_short[return_short['TICKER'].isin(top_list)]
            ttt_top =return_short[return_short['TICKER'].isin(top_list)]
            ttt_bottom =return_short[return_short['TICKER'].isin(bottom_list)]
            ttt_top =ttt_top[ttt_top['year']==year]
            for yeardate in set(ttt_top['date']):
                temp_top = ttt_top[ttt_top['date']==yeardate]
                temp_bottom = ttt_bottom[ttt_bottom['date']==yeardate]
                summkt_top=temp_top['MKT'].sum()
                summkt_bottom=temp_bottom['MKT'].sum()
                if summkt_top !=0:
                    vwrt_top=sum(temp_top['RET']*temp_top['MKT'])/summkt_top
                    ewrt_top=temp_top['RET'].mean()
                else:
                    vwrt_top=0
                    ewrt_top=temp_top['RET'].mean()
                if summkt_bottom !=0:
                    vwrt_bottom=sum(temp_bottom['RET']*temp_bottom['MKT'])/summkt_bottom
                    ewrt_bottom=temp_bottom['RET'].mean()
                else:
                    vwrt_bottom=0
                    ewrt_bottom=temp_bottom['RET'].mean()
                vwrt=vwrt_top-vwrt_bottom
                ewrt=ewrt_top-ewrt_bottom
                temp_dict_forstate.append([yeardate,vwrt,ewrt,vwrt_top,ewrt_top,vwrt_bottom,ewrt_bottom,state])
        else:
            pass
    gc.collect()
return_list= pd.DataFrame(temp_dict_forstate,columns=["yeardate","vwrt","ewrt",'vwrt_top','ewrt_top','vwrt_bottom','ewrt_bottom',"state"])
   



KeyboardInterrupt: 

In [12]:
#return_list=return_list.fillna(0)
#return_list.to_csv('ReturnFileGroupbyState.csv')
return_list =pd.read_csv('ReturnFileGroupbyState.csv')
return_list=return_list[['yeardate',
 'vwrt',
 'ewrt',
 'vwrt_top',
 'ewrt_top',
 'vwrt_bottom',
 'ewrt_bottom',
 'state']]
return_list

,yeardate,vwrt,ewrt,vwrt_top,ewrt_top,vwrt_bottom,ewrt_bottom,state
0,19760130,0.085106,0.000000,0.085106,0.042553,0.000000,0.000000,WA
1,19760227,-0.102474,0.000000,-0.102474,-0.053922,0.000000,0.000000,WA
2,19761029,0.040854,0.000000,0.040854,-0.091428,0.000000,0.000000,WA
3,19761130,-0.001018,0.000000,-0.001018,-0.014162,0.000000,0.000000,WA
4,19760331,0.134436,0.000000,0.134436,0.070329,0.000000,0.000000,WA
5,19760930,0.218435,0.000000,0.218435,0.111888,0.000000,0.000000,WA
6,19760430,0.102509,0.000000,0.102509,0.053398,0.000000,0.000000,WA
7,19761231,0.026548,0.000000,0.026548,0.130989,0.000000,0.000000,WA
8,19760528,-0.062672,0.000000,-0.062672,-0.057220,0.000000,0.000000,WA
9,19760630,0.052532,0.000000,0.052532,0.001275,0.000000,0.000000,WA


In [8]:
law='law_date.csv'
law=pd.read_csv('law_date.csv')
date = set(return_list['yeardate'])
law=law.fillna(0)
law.drop(law.columns[[0]],1,inplace=1)

In [23]:
lawname = list(law.columns)
states =set(law['State'])
date=list(set(return_list['yeardate']))
date_temp=copy.deepcopy(date)
datelen=len(list(set(return_list['yeardate'])))
temp_state_list=[]
for state in states:
#if 1==1:
    #state="IA"
    w=law[law['State']==state]
    r =TransformDataFrame(w)         
    zipped_r =CombineShortValue(r,date)
    #print zipped_r
    #print zipped_r.keys()
    #print len(zipped_r[19891231])
    law_status_list=[]
   
    for sr in r:
        #print r[7]
        #print len(zipped_r[r[7]])
        
        law_status_list.append(zipped_r[sr[0]])
        #print law_status_list
        #print zipped_r[sr[0]]
        #print len(law_status_list[0])
        
   
    law_status_list[7]
    #print r[7]
    tem_law = pd.DataFrame(law_status_list,index=lawname).T
    #print pd.DataFrame(law_status_list,index=lawname)
   
    
    try:
        
        #print len(sorted(date))
        tem_law['date'] = sorted(date)
        #print tem_law
    except Exception as e:
        #print tem_law.head(492)
        tem_law= tem_law.head(492)
        tem_law['date'] = int(sorted(date))
    
    temp_state_list.append(tem_law)
#print len(temp_state_list)
#print temp_state_list[0]
lawname.append('date')


df_empty =pd.DataFrame(columns=list(lawname))
for ssss in temp_state_list:
    df_empty = pd.merge(df_empty,ssss[list(lawname)],how="outer")
df_empty['date'] =df_empty['date'].astype(np.int64)

df_empty.pd.to_csv('law.csv')



AttributeError: 'DataFrame' object has no attribute 'pd'

In [395]:


print "return columns:",len(set(return_list['yeardate']))
print "law columns:",tem_law.shape[0]
print set(states)

return columns: 492
law columns: 492
set(['WA', 'DE', 'WI', 'WV', 'HI', 'FL', 'WY', 'NH', 'NJ', 'NM', 'TX', 'LA', 'NC', 'ND', 'NE', 'TN', 'NY', 'PA', 'AK', 'NV', 'VA', 'CO', 'CA', 'AL', 'AR', 'VT', 'IL', 'GA', 'IN', 'IA', 'MA', 'AZ', 'ID', 'CT', 'ME', 'MD', 'OK', 'OH', 'UT', 'MO', 'MN', 'MI', 'RI', 'KS', 'MT', 'MS', 'SC', 'KY', 'OR', 'SD'])


In [496]:

def TransformDataFrame(w):
    collected=[]
    for information in list(w.T[list(w.T.columns)[0]]):
        #print information
        
        if  information==0:
            collected.append((0,0))
        elif len(information)==2:
            collected.append((information,"State"))
        elif isinstance(information,str):
            splitout =[x for x in information.split(' ') if x !='']
            #print splitout
            date =TrandsformYMD(splitout[0])
            if len(splitout)==1:
                collected.append((int(date),"NOW"))
            elif len(splitout)==2:
                
                try:
                    tttemp =int(splitout[1])
                
                    collected.append((int(date),tttemp))
                
                except Exception as e:
                    print splitout[1]
                    collected.append((int(date),splitout[1]))
            
                    
            elif len(splitout)>=3 and splitout[1] =="(Y)" and len(splitout[2:])==1:
                #print int(TrandsformYMD(splitout[2].split("(")[0]))

                collected.append((int(date),int(TrandsformYMD(splitout[2].split("(")[0]))))
            elif len(splitout)>=3  and len(splitout[2:])>1:
                t=[(int(date),"Y")]

                for date2_str in splitout[2:]:


                    if isinstance(date2_str,str):

                        if "(" in date2_str:
                            date2 = TrandsformYMD(date2_str.split("(")[0])


                            date2_status=date2_str.split("(")[1].strip(")")

                            te =(int(date2),date2_status)

                            t.append(te)
                    else:
                        pass

                collected.append(t)


    return collected 
w =law[law['State']=="IA"]
print list(w.T[list(w.T.columns)[0]])
r =TransformDataFrame(w)
r

['IA', '07/01/1997', 0, 0, 0, '12/31/1989', 0, '3/23/2011 (Y) 12/14/2014(N)', 0, 0, 0, 0, '12/31/1989']


[('IA', 'State'),
 (19970701, 'NOW'),
 (0, 0),
 (0, 0),
 (0, 0),
 (19891231, 'NOW'),
 (0, 0),
 (20110323, 20141214),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (19891231, 'NOW')]

In [88]:
a=[1,0.4,1,2,3]
a.index(2)
a[:a.index(2)+1]

[1, 0.4, 1, 2]

In [189]:

a=list(return_short['RET'])
a.index("B")

1377

In [133]:
law

,State,Business Combination,Fair Price,Control Share Acquisition,Control Share Cash-out,Poison Pill Statute,Disgorgement,Mandatory Staggered board,Anti Greenmail,Golden Parachute Restriction,Tin Parachute Blessing,Assumption of Labor Contracts,Expanded Constituency
0,AL,0,0,0,0,0,0,0,0,0,0,0,0
1,AK,0,0,0,0,0,0,0,0,0,0,0,0
2,AZ,7/22/1987 (Combo),0,7/22/1987,0,01/01/1996,0,0,7/22/1987,7/22/1987,0,0,7/22/1987
3,AR,0,0,0,0,0,0,0,0,0,0,0,0
4,CA,0,0,0,0,0,0,0,0,0,0,0,0
5,CO,0,0,0,0,3/31/1989,0,0,0,0,0,0,0
6,CT,06/07/1988,6/4/1984,0,0,01/01/1997,0,0,0,0,0,0,06/07/1988
7,DE,02/02/1988,0,0,0,0,0,0,0,0,0,04/08/1988,0
8,FL,0,07/02/1987,07/02/1987,0,07/01/1990,0,0,0,0,0,0,6/27/1989
9,GA,03/03/1988,07/01/1985,0,0,07/01/1989,0,0,0,0,0,0,07/01/1989


In [84]:
a =pd.DataFrame({1:[2,2,1,1,1],2:[3,2,1,1,1]}).reset_index()
list(a['index'][a[1]==1])
a.drop(list(a['index'][a[1]==1]))

,index,1,2
0,0,2,3
1,1,2,2
2,2,1,1
3,3,1,1
4,4,1,1


In [86]:
list(a['index'][a[1]==1])
a.drop(list(a['index'][a[1]==1]))

[2, 3, 4]

In [87]:
a.drop(list(a['index'][a[1]==[1,2]]))

,index,1,2
0,0,2,3
1,1,2,2


In [326]:
'20101002'<'20100203'

False

In [97]:
a[a[1]==[2,1]]

TypeError: invalid type comparison

In [136]:
a =pd.DataFrame({1:[2,2.0,1,1,1],2:[3,2,1,1,1]}).reset_index()
a

,index,1,2
0,0,2.0,3
1,1,2.0,2
2,2,1.0,1
3,3,1.0,1
4,4,1.0,1


In [167]:
a =pd.DataFrame({1:[2,2,1,1,1],2:[3,2,1,1,1]})
se=a[1]
se.str.isnumeric()

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [99]:
a['return']=a[1][a[1]==2]/a[1][a[1]==2].shift(1)-1
a

,index,1,2,return
0,0,2,3,NaN
1,1,2,2,0.0
2,2,1,1,NaN
3,3,1,1,NaN
4,4,1,1,NaN


In [102]:
a['return']=a[1][a[1]==1]/a[1][a[1]==1].shift(1)-1

In [69]:
a[a[2]==]

TypeError: invalid type comparison

In [19]:
set([1,2])|set([2,3])

{1, 2, 3}

In [199]:
a=[[1,2],[2,3]]
a


[[1, 2], [2, 3]]

In [202]:
pd.DataFrame(a,columns=["w","y"])

,w,y
0,1,2
1,2,3


In [147]:
filter(NULL,[1,2,3,'','',0])

NameError: name 'NULL' is not defined

In [154]:
a =[1,2,3,'','',0]
print [aw for aw in a if aw != '']

[1, 2, 3, 0]


In [151]:
a

[1, 2, 3, '', 0]

In [236]:
(0,0)==(0,0)

True

In [241]:
[1,2]+[2,3]

[1, 2, 2, 3]

In [315]:
a= [(19760130, 19880408, 0), (19880408, 19950425, 1), (19950425, 20070307, 0), (20070307, 20161230, 1)]


ValueError: dictionary update sequence element #0 has length 3; 2 is required

In [317]:
target =20000101
def returendict(a):
    a_dict={}
    for aa in a:
        a_dict.update({aa[0]:aa})
    return a_dict
    

def returnvalue(target,a_dict):
    test = a_dict.keys()
    
    test.insert(0,target)
    test.sort()

    return a_dict[sorted(a_dict.keys())[test.index(target)-1]][2]

print returnvalue(target,a_dict)


0
